<img style="float: left;" src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/ligo-virgo-300x120.png?raw=true"> 

# Handling gravitational-wave sky maps with Multi-Order Coverage

This document explains how gravitational wave sky maps can be easily and efficiently visualized and processed using [Multi-Order Coverage (MOC)](http://ivoa.net/documents/MOC/20140602/REC-MOC-1.0-20140602.pdf) maps based on [HEALPix](http://healpix.sourceforge.net/) sky tessellation. 
We compute the MOC region at a given probability level and subsequently, we simultaneously query databases for retrieving objects whose position falls within this MOC map. If databases are adapted to supporting MOC based queries, they will offer then a useful method allowing any kind of sky region query.

For this tutorial we use the simulated sky maps from [**The First Two Years of Electromagnetic Follow-Up with Advanced LIGO and Virgo**](http://arxiv.org/abs/1404.5623) for compact binary Coalescence (CBC) sources. The sky maps for burst events are simulated from [**Localization of short duration gravitational-wave transients with the early advanced LIGO and Virgo detectors**](http://arxiv.org/abs/1409.2435).
A gallery of these sky maps is shown [here](https://losc.ligo.org/s/skymapViewer/skymaps.html).

We provide sample code in Python; you can download this document and run the code samples in [IPython Notebook](http://ipython.org/notebook.html) (now known as the Jupyter Notebook). The results are displayed in real time in [Aladin Sky Atlas](http://aladin.u-strasbg.fr/) which is embedded in the document.

 

## Table of Contents

1. [Installation and initialization](#1.-Installation-and-initialization)
    1. [Sky map visualization with Aladin](#1.A-Sky-map-visualization-with-Aladin)
    1. [Python packages](#1.B-Python-packages)
1. [Multiscale meshes of gravitational-wave sky maps using MOC](#2.-Multiscale-meshes-of-gravitational-wave-sky-maps-using-MOC)
     1. [CBC sky maps](#2.A-CBC-sky-maps)
     1. [Event id 18951](#2.B-Event-id-18951)
1. [Query Catalogs from MOCs](#3.-Query-Catalogs-from-MOCs)
    1. [Query a single catalog](#3.A-Query-a-single-catalog)
    1. [Ranked list of galaxies in 3D sky map](#3.B-Ranked-list-of-galaxies-in-3D-sky-map)
    1. [Queries running simultaneously](#3.C-Queries-running-simultaneously) 
1. [Operation between MOC maps](#4.-Operation-between-MOC-maps)
    1. [Intersection between MOC skymaps and VizieR tables footprints](#4.A-Intersection-between-MOC-skymaps-and-VizieR-tables-footprints)
1. [Utility](#5.-Utility)
    1. [Interactive MOCs](#5.A-Interactive-MOCs)

## 1. Installation and initialization


### 1.A Sky map visualization with Aladin
The interoperability between the Aladin Sky Atlas and the Python function outputs is obtained using [SAMP](http://www.ivoa.net/documents/SAMP/) (Simple Application Messaging Protocol). The best performance is achieved by installing Aladin Desktop and the Aladin java applet is embedded in the notebook. 
Anyhow, a detach button is supplied to disjoin the Aladin window. 

As the code runs, the results are displayed in real time in the Aladin planes. **Before running the notebook check the SAMP connection: Aladin bar --> Interop --> Connect with SAMP.** 

In [ ]:
from IPython.core.display import HTML
HTML('<iframe src=http://aladin.u-strasbg.fr/java/nph-aladin.pl width=770 height=930></iframe>')

### 1.B Python packages

We will need [healpy](https://healpy.readthedocs.org/en/latest/) for reading the probability sky map files and [mocpy](https://github.com/tboch/mocpy) for parsing and manipulating MOCs.

For this tutorial, we will also use the [astropy.vo](http://astrofrog-debug.readthedocs.org/en/latest/vo/index.html) astropy subpackage for accessing to the Virtual Observatory (VO) services. The [astropy.utils](http://astrofrog-debug.readthedocs.org/en/latest/utils/) subpackage contains general-purpose utility functions, [astropy.table](http://docs.astropy.org/en/stable/table/) provides functionality for storing and manipulating heterogeneous tables, [numpy](http://www.numpy.org/) for scientific computing with Python, [urlparse](https://docs.python.org/2/library/urlparse.html) is a standard interface to break Uniform Resource Locator (URL) strings up in components (the urlparse module is renamed to urllib.parse in Python 3) and  [future](https://pypi.python.org/pypi/future) for running under Python 2 and Python 3. The json files - in which the contour lines are saved - are manipulated using [json](https://docs.python.org/2/library/json.html) package. The last section makes use of [ipywidgets](https://pypi.python.org/pypi/ipywidgets); they are interactive HTML widgets for Jupyter notebooks and the IPython kernel.

SAMP is a protocol that is used by a number of other tools such as [TOPCAT](http://www.star.bris.ac.uk/~mbt/topcat/), [SAO Ds9](http://ds9.si.edu/site/Home.html), and [Aladin](http://aladin.u-strasbg.fr/); it is possible to send and receive data to and from these tools.
Here the [Aladin script commands](http://aladin.u-strasbg.fr/java/AladinScriptManual.gml) are converted in Python strings and sent via [SAMPIntegratedClient class](http://jakevdp.github.io/astropy/api/astropy.vo.samp.SAMPIntegratedClient.html#astropy.vo.samp.SAMPIntegratedClient). 

In [ ]:
from __future__ import print_function

# Sending Aladin script commands

def send_script( script ):
    
    """
    
    Sending script commands to Aladin via SAMP
    
    """
    
    from astropy.vo.samp import SAMPIntegratedClient
     
    client = SAMPIntegratedClient()
    client.connect()

    params = {}
    message = {} 
    message[ "samp.mtype" ] = "script.aladin.send"
    message[ "samp.params" ] = { "script" : script }  

    client.notify_all( message )

    client.disconnect()

    
def cview( url ): 
    
    """
    
    Creation of view: url
    
    """
    cview_url = 'cview' + ' ' + url
    send_script( cview_url )

    
def rename ( plane ):
    
    """
    
    Rename plane

    """
    
    rename_plane = 'rename' + ' ' + plane
    send_script( rename_plane )
        
        
def get_hips ( catalog ): 
    
    """
    
    Call a remote image or tabular data server
    
    """

    hips = 'get' + ' ' + 'hips(' + catalog + ')'
    send_script ( hips )
    

def cview_plane ( plane ):
    
    """
    
    Creation of view: plane
      
    """
    
    show_plane = 'cview'  + ' "'+plane+'"'
    send_script ( show_plane )
    
    
def hide ( plane ):
    
    """
    
    Hide plane
     
    """
    
    hide_plane = 'hide'  + ' '+ plane
    send_script ( hide_plane )
    
    
def draw_line ( line_values ):
    
    """
    
    Graphical overlay commands: drawing a line
    
    """
    
    draw_line = 'draw' + ' ' + 'line' + ' ' + line_values
    send_script( draw_line )


def draw_newtool (name):
    
    """
    
    Graphical overlay commands:
    creating  a new drawing plane

    """
    
    draw_newtool = 'draw' + ' ' + 'newtool' + ' ' + name
    send_script( draw_newtool )
    
    
def rm_all():
    
    """
    
    Removing all planes
    
    """
    
    rm_all = 'rm -all'
    send_script( rm_all )
    
    
def send_file( infile ):
    
    """
    
    Sending file/table to Aladin plane via SAMP
        
    """

    global params
     
    from astropy.vo.samp import SAMPIntegratedClient
     
    client = SAMPIntegratedClient()
    client.connect()
    params = {}
    
    import sys
    import os.path
    
    if sys.version > '3':
        import urllib.parse
        params[ "url" ] = urllib.parse.urljoin( 'file:', os.path.abspath( infile ) )
    else:
        import urlparse
        params[ "url" ] = urlparse.urljoin( 'file:', os.path.abspath( infile ) )

    message = {}
    message[ "samp.mtype" ] = "image.load.fits"
    message[ "samp.params" ] = params

    client.notify_all( message )
    client.disconnect()

The functions *get_json_link* and *plot_contours_from_json*  are dedicated to manipulate .json files containing the contour plots of a probability sky map issued by LVC. *MOC_confidence_region*  is written to extract a MOC map setting a certain level of probability.

In [ ]:
def get_json_link( json_link ):

    """
    
    Plotting contour lines from a specific url
    
    """
    
    import json
    import sys
    import os.path
    
    # download the json file from "url" and save it locally under "contour.json"
    if sys.version < '3':
        import urllib
        jsonfile = urllib.URLopener()
        jsonfile.retrieve( json_link, "contour.json" )
    
    else:
        import urllib.request
        urllib.request.urlretrieve( json_link, "contour.json" )
     
    with open( 'contour.json' ) as data_file:
       data = json.load( data_file )

    contour_pieces = len( data[ 'contours' ] )

    percentile = ('10-percentile','20-percentile','30-percentile','40-percentile',
                  '50-percentile','60-percentile','70-percentile',
                  '80-percentile','90-percentile')

    for percentile_json in percentile:
       draw_newtool ( percentile_json )
       plot_contours_from_json( data, contour_pieces, percentile_json )
        
        
def plot_contours_from_json( data, contour_pieces, percentile_json ):

    """
    
    Managing the contour lines in a LVC json file
        
    """
    
    i = 0
    for i in range( 0, contour_pieces ):
        contour = data[ 'contours' ][ i ]
        percentile = contour[ 'name' ]

        if percentile == percentile_json:
            values = contour[ 'coords' ]

            # sending Aladin plane
            line = ( str( values ).replace('[' , '' ).replace(']' , '') )
            draw_line ( line )


def MOC_confidence_region( infile, percentage, short_name = ' ' ):
      
    """
    
    Multi-Order coverage map (MOC) of sky area enclosed within a contour plot
    at a given confidence level.
    
    Input:
         infile: healpix format
                 LVC probability sky map
         percentage: float
                  probability percentage of the enclosed area  
         short_name: str
                 output file name
     
    Output: fits format
                 MOC map named "short_name"_"percentage" 
                 
                 Remark: for json format change the statement
                 "moc.write(short_name+'_MOC_'+str(percentage), format='fits' )" -->  
                 "moc.write(short_name+'_MOC_'+str(percentage), format='json' )"        

    """
 
    import healpy as hp
    import numpy as np
     
    #reading skymap
    hpx = hp.read_map( infile, verbose = False )
    npix = len( hpx )
    nside = hp.npix2nside( npix )
 
    sort = sorted( hpx, reverse = True )
    cumsum = np.cumsum( sort )
    index, value = min( enumerate( cumsum ), key = lambda x: abs( x[1] - percentage ) )

    # finding ipix indices confined in a given percentage 
    index_hpx = range( 0, len( hpx ) )
    hpx_index = np.c_[ hpx, index_hpx ]

    sort_2array = sorted( hpx_index, key = lambda x: x[0], reverse = True )
    value_contour = sort_2array[ 0:index ]

    j = 1 
    table_ipix_contour = [ ]

    for i in range ( 0, len( value_contour ) ):
        ipix_contour = int( value_contour[i][j] )
        table_ipix_contour.append( ipix_contour )
          
     
    # from index to polar coordinates
    theta, phi = hp.pix2ang( nside, table_ipix_contour )

    # converting these to right ascension and declination in degrees
    ra = np.rad2deg( phi )
    dec = np.rad2deg( 0.5 * np.pi - theta )


    # creating an astropy.table with RA[deg] and DEC[deg] ipix positions
    from astropy.table import Table
    contour_ipix = Table([ ra, dec ], names = ('RA[deg]', 'DEC[deg]'), 
                         meta = {'ipix': 'ipix table'})
     
    
    # setting MOC order
    from math import log
    moc_order = int( log( nside, 2 ) )

    # creating a MOC map from the contour_ipix table
    moc = MOC.from_table( contour_ipix, 'RA[deg]', 'DEC[deg]', moc_order )

    # writing MOC file in fits
    moc.write( short_name + '_MOC_' + str( percentage ), format = 'fits' )

    # sending to Aladin plane
    send_file( short_name + '_MOC_' + str( percentage ) )
    cview( url = str( params[ 'url' ]) )
    rename ( plane = short_name  + '_MOC_' + str( percentage ) )

## 2. Multiscale meshes of gravitational-wave sky maps using MOC
MOC is a multiscale mapping based on HEALPix sky tessellation. It is essentially a simple way to map irregular and complex sky regions into hierarchically grouped predefined cells.
Each MOC cell is defined by two numbers: the hierarchy level (HEALPIX ORDER) and the pixel
index (HEALPIX NPIX).The NUNIQ scheme defines an algorithm for packing an (ORDER, NPIX)
pair into a single integer for compactness:

$$uniq = 4\times 4^{(order)} + npix$$


** Caution. ** By reducing a map in only a single confidence region, the probability distribution within that region is irreversible lost; see also [Essick et al. (2015)](http://adsabs.harvard.edu/abs/2015ApJ...800...81E).

### 2.A CBC sky maps 
The sky maps for compact binary Coalescence (CBC) are simulated from two analysis pipelines: the rapid pipeline - *Bayestar* - and the computationally intensive pipelines *Lalinference_ MCMC* or *Lalinference_Nest*.

The rapid Bayesian position reconstruction code that will produce
accurate sky maps less than a minute after any BNS merger detection.  The
LALINFERENCE_MCMC (van der Sluys et al. [2008b](http://adsabs.harvard.edu/abs/2008ApJ...688L..61V); Raymond et al. [2009](http://adsabs.harvard.edu/abs/2009CQGra..26k4007R)), LALINFERENCE_NEST (Veitch & Vecchio [2010](http://adsabs.harvard.edu/abs/2010PhRvD..81f2003V)), and LALINFERENCE_BAMBI (Graff et al. [2012](http://adsabs.harvard.edu/abs/2012MNRAS.421..169G), [2013](http://mnras.oxfordjournals.org/content/441/2/1741)) stochastic samplers were also used to follow up a subset of detected GW events. Though these analyses are significantly more computationally costly than BAYESTAR, taking  hours  to  days,  they  can  provide  improved  sky  location estimates when the GW signal is very weak in one detector, and also yield not just sky localization but the full multidimensional probability distribution describing the parameters of a circularized compact binary merger.

### 2.B Event id 18951
This is the event 18951 - LIGO Hanford and Livingston joint detection. It simulates the response from a binary neutron star event at 75 Mpc. Here, we map the sky region in which the 90% of probability is enclosed. The MOC region is shown in white shadow over a discrete set of contour plots; each line encloses a given percentage of probability level from 10% to 90% in step of 10% ([more about the contour plots](https://losc.ligo.org/s/skymapViewer/contours.html)).

In [ ]:
# selecting an event id (2015);
# http://www.ligo.org/scientists/first2years/
event_id = '18951'

# bayestar sky map
skymap_pipeline = 'bayestar'

# setting enclosed probability percentage 
prob_percentage = 0.9

# loading the simulated CBC event id (2015)
from astropy.utils.data import download_file

url_id = 'http://www.ligo.org/scientists/first2years/2015/compare/'+event_id+'/'+skymap_pipeline+'.fits.gz'
pipeline_event = download_file( url_id, cache = True, timeout = 300 )

# sending to Aladin plane
send_file ( pipeline_event )
rename ( skymap_pipeline + event_id )

# plotting contours from a specific url
from mocpy import MOC
get_json_link( 'https://losc.ligo.org/s/skymapViewer/json/skymaps/F2Y/'+event_id+'.json' )

# MOC extraction: 
#        area enclosed within a specific contour plot at a given confidence level
MOC_confidence_region( infile = pipeline_event, percentage = prob_percentage, 
                      short_name = skymap_pipeline + event_id)

# loading the MOC file
MOC_file = MOC.from_file( skymap_pipeline + event_id + '_MOC_' + str(prob_percentage) )

# square degrees in a whole sphere
from math import pi
square_degrees_sphere = (360.0**2)/pi

# printing area
area_sq2 = round( ( MOC_file.sky_fraction * square_degrees_sphere ), 1 )
print ( str( int( prob_percentage*100 ) )+'%' + ' area = ', area_sq2, 'sq. deg' )

# loading DSS colored for sky background
get_hips( "P/DSS2/color" )

<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/id18951.gif?raw=true" width="600" height="600"> 
**Blink image between the contour plot and the MOC extraction of 90% area. See the completed and updated results in the section [Sky map visualization with Aladin](#1.A-Sky-map-visualization-with-Aladin)**.

In [ ]:
# removing all planes
rm_all()

# 3. Query Catalogs from MOCs
In this section, we show how MOC maps can be use to query catalog objects that falls into the sky map region. The MOCs of all VizieR tables footprints are available [on line](http://alasky.u-strasbg.fr/footprints/tables/vizier/) (about 16.000 tables) and can be queried simultaneously in few seconds.

### 3.A Query a single catalog 

As an example, we query the [Gravitational Wave Galaxy Catalog](https://arxiv.org/abs/1103.0695) from the MOC map obtained in section [2.B Event id 18951](#2.B-Event-id-18951). The source positions are displayed in red inside the MOC region drawn in white.


In [ ]:
catalog = 'VII/267/gwgc' # selecting catalog
catalog_renamed = catalog.replace('/', '_')

# selecting MOC coverage
from mocpy import MOC
moc = MOC.from_file( 'bayestar18951_MOC_0.9' )

# sending to Aladin plane
send_file( 'bayestar18951_MOC_0.9')
rename ( 'bayestar18951_MOC_0.9' )

# querying from MOC ignoring astropy.io.votable.exceptions
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    table = moc.query_vizier_table( catalog, max_rows = 100000 ) 

# file output: votable format
table.write( catalog_renamed + 'MOC_query', format = 'votable', overwrite = True )

# sending to the Aladin plane 
send_file( catalog_renamed + 'MOC_query' )

# loading DSS colored for sky background
get_hips( "P/DSS2/color" )

<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/query_single_catalog.jpg?raw=true" width="600" height="600"> 
**Catalog query from a MOC probability map (90% conf. level). See the completed and updated results in the section [Sky map visualization with Aladin](#1.A-Sky-map-visualization-with-Aladin)**

In [ ]:
# removing all planes
rm_all()

### 3.B Ranked list of galaxies in 3D sky map
Singer et al. ([2016](https://arxiv.org/pdf/1603.07333v2.pdf)) discuss a rapid algorithm for obtaining joint three-dimensional estimates of sky location and luminosity distance from observations of binary neutron star mergers with Advanced LIGO and Virgo.   
They argued that combining the reconstructed volumes with positions and redshifts of possible host galaxies can provide a manageable list of targets to search for optical or infrared emission. The 2MASS Redshift Survey (2MRS) (Huchra et al. [2012](http://arxiv.org/abs/1108.0669)) is downloaded for this purpose.
In order to reduce the query time, the 2MRS is directly queried from the MOC region *e.g* in which the 90% of probability is enclosed. Here, the MOC map obtained in section [2.B Event id 18951](#2.B-Event-id-18951) is used.

See [here](https://arxiv.org/pdf/1605.04242v1.pdf) for more details on the sample python code provided in the online supplement to the Letter [**GOING THE DISTANCE: MAPPING HOST GALAXIES OF LIGO AND VIRGO SOURCES IN THREE DIMENSIONS USING LOCAL COSMOGRAPHY AND TARGETED FOLLOW-UP.**](https://arxiv.org/pdf/1603.07333v2.pdf)

In [ ]:
# downloading 3D HEALPix sky map
from astropy.utils.data import download_file
url = ('http://asd.gsfc.nasa.gov/Leo.Singer/'+'going-the-distance/2015/compare/18951/'+'bayestar.fits.gz')
filename = download_file(url, cache=True)

# reading HEALPix layers
import healpy as hp
prob, distmu, distsigma, distnorm = hp.read_map(filename, 
                                                field=[0, 1, 2, 3], verbose=False)

# HEALPix resolution 
npix = len(prob)
nside = hp.npix2nside(npix)

pixarea = hp.nside2pixarea(nside)

# Ranking list of galaxies from a MOC region
from mocpy import MOC
moc = MOC.from_file( 'bayestar18951_MOC_0.9' )

# sending to Aladin plane
send_file( 'bayestar18951_MOC_0.9')
rename ( 'bayestar18951_MOC_0.9' )

catalog = 'J/ApJS/199/26/table3'  # 2MASS Redshift Survey 

# querying from MOC ignoring astropy.io.votable.exceptions
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    cat = moc.query_vizier_table( catalog, max_rows = 100000 )

import numpy as np
from scipy.special import gammaincinv

completeness = 0.5
alpha = -1.0
MK_star = -23.55

MK_max = MK_star + 2.5*np.log10(gammaincinv(alpha + 2, completeness))

# selecting only galaxies with positive redshifts and absolute
# magnitudes greater than M(max)
from astropy.cosmology import WMAP9 as cosmo

import astropy.units as u
import astropy.constants as c

z = (u.Quantity(cat['cz']) / c.c).to(u.dimensionless_unscaled)

MK = cat['Ktmag'] - cosmo.distmod(z)
keep = (z > 0) & (MK < MK_max)

cat = cat[keep]
z = z[keep]

# luminosity distance and HEALPix index of each galaxy
r = cosmo.luminosity_distance(z).to('Mpc').value

theta = 0.5*np.pi - cat['_DEJ2000'].to('rad').value
phi = cat['_RAJ2000'].to('rad').value
ipix = hp.ang2pix(nside, theta, phi)

# probability density per unit volume at the position of each galaxy
from scipy.stats import norm
dp_dV = prob[ipix]*distnorm[ipix]*norm(distmu[ipix], distsigma[ipix]).pdf(r) / pixarea

#sorting the galaxies by descending probability density
galaxies_in_moc = cat[np.flipud(np.argsort(dp_dV))][:]

# adding probability galaxy position to the catalog
from astropy.table import Column

dp_dV_sort = np.flipud(np.argsort(dp_dV))[:]
dp_dV_value = dp_dV[dp_dV_sort]

# rounding
dp_dV_value_round = []
dp_dV_value_round = ['{:.3e}'.format(i) for i in dp_dV_value]

probability_galaxy_position = Column(dp_dV_value_round, name = 'dp_dV')

galaxies_in_moc.add_column(probability_galaxy_position, index=0)
print (galaxies_in_moc['_RAJ2000', '_DEJ2000', 'Ktmag','dp_dV'])

# sending to Aladin plane the weighted catalog 
galaxies_in_moc.write( 'ranked_list_galaxies', format = 'votable', overwrite = True )
send_file( 'ranked_list_galaxies' )

# loading DSS colored for sky background
get_hips( "P/DSS2/color" )

<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/%20Ranked3Dsky%20map.png?raw=true" width="600" height="600"> 
**2MASS Redshift Survey catalog; in the first column the probability density per unit volume at the position of each galaxy is reported. See the completed and updated results in the section [Sky map visualization with Aladin](#1.A-Sky-map-visualization-with-Aladin)**

In [ ]:
# removing all planes
rm_all()

### 3.C Queries running simultaneously

Here we simultaneously query 6 VizieR tables footprints. The following catalogs are selected:
**a**) Gravitational Wave Galaxy Catalogue (White [2011](https://arxiv.org/abs/1103.0695)); **b**) Compact Binary Coalescence Galaxy Catalog (Kopparapu et al., [2008](http://iopscience.iop.org/article/10.1086/527348/meta;jsessionid=B788E08A98425B3F35D19E56C6ABA2F3.c1.iopscience.cld.iop.org)); **c**) Catalogue of Rich Clusters of Galaxies (Abell et al., [1989](http://cdsads.u-strasbg.fr/cgi-bin/nph-bib_query?1989ApJS...70....1A&db_key=AST&nosetcookie=1)); **d**) Northern Cluster Catalog (Gal et al., [2009](http://iopscience.iop.org/article/10.1088/0004-6256/137/2/2981/meta)); **e**) MCXC Meta-Catalogue X-ray galaxy Clusters (Piffaretti et al., [2011](http://www.aanda.org/component/article?access=bibcode&bibcode=2011A%252526A...534A.109P)); **f**) 2MASS Redshift Survey (Huchra et al. [2012](http://arxiv.org/abs/1108.0669))
Here the MOC map obtained in section [2.B Event id 18951](#2.B-Event-id-18951) is used.

In [ ]:
# selecting catalogs
catalogs = ['VII/267/gwgc','J/ApJ/675/1459/table1','VII/110A',
            'J/AJ/137/2981','J/A+A/534/A109','J/ApJS/199/26/table3'] 

# selecting MOC coverage
from mocpy import MOC
moc = MOC.from_file( 'bayestar18951_MOC_0.9') 

# sending to Aladin plane
send_file( 'bayestar18951_MOC_0.9')
rename ( 'bayestar18951_MOC_0.9' )

# querying from MOC ignoring astropy.io.votable.exceptions
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for catalog in catalogs:
        print ()
        table = moc.query_vizier_table( catalog, max_rows = 100000 ) 
        print (table)
        catalog_renamed = catalog.replace('/', '_')
        table.write( catalog_renamed + 'MOC_query', format = 'votable', overwrite = True )
        send_file( catalog_renamed + 'MOC_query' )
        
# loading DSS colored for sky background
get_hips( "P/DSS2/color" )

<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/queries_simultaneously.jpg?raw=true" width="600" height="600"> 
**Multi-catalog query from a MOC probability map (90% conf. level). See the completed and updated results in the section [Sky map visualization with Aladin](#1.A-Sky-map-visualization-with-Aladin)**

In [ ]:
# removing all planes
rm_all()

# 4. Operation between MOC maps

### 4.A Intersection between MOC skymaps and VizieR tables footprints

The operations between the MOC maps (union, intersection, subtraction, difference) are extremely simple and fast (generally a few milliseconds) even for very complex sky regions.
Here the intersection between the DSS coverage and the MOC sky map in [2.B Event id 18951](#2.B-Event-id-18951) is shown. 

In [ ]:
# selecting MOC coverage
from mocpy import MOC
moc_1 = MOC.from_file( 'bayestar18951_MOC_0.9' ) 

# sending to Aladin plane
send_file( 'bayestar18951_MOC_0.9')
rename ( 'bayestar18951_MOC_0.9' )

# loading the MOC coverage map of SDSS Photometric Catalog (9)
from astropy.utils.data import download_file
url_id = 'http://alasky.u-strasbg.fr/footprints/tables/vizier/V_139_sdss9/MOC'
sdss9 = download_file( url_id, cache = True, timeout = 300 )
send_file( sdss9 )
rename ( 'sdss9_MOC' )

#load sdss9 MOC coverage
moc_2 = MOC.from_file( sdss9 ) 

# Intersection operation and writing file
inter = moc_1.intersection( moc_2 )
inter.write( 'inter', format = 'fits')

#sending to Aladin plane
send_file( 'inter' )
rename ( 'inter' )

# loading DSS colored for sky background
get_hips( "P/DSS2/color" )

<img src="https://github.com/ggreco77/Multi-Order-Coverage-of-probability-skymaps/blob/master/intersection_sdss9_id18951.jpg?raw=true" width="600" height="600"> 
**Intersection (in red) between the MOC map of SDSS Photometric Catalog (in blue) and MOC sky map at the 90% probability of event id 18951 (in white). See the completed and updated results in the section [Sky map visualization with Aladin](#1.A-Sky-map-visualization-with-Aladin)**

In [ ]:
# removing all planes
rm_all()

# 5. Utility  

### 5.A Interactive MOCs
The probability MOC regions are automatically generated by adjusting the probability threshold slider. The [2.B Event id 18951](#2.B-Event-id-18951) MOC sky map is choosen in the example. 

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, fixed

# selecting an event id (2015);
# http://www.ligo.org/scientists/first2years/
event_id = '18951'

# bayestar sky map
skymap_pipeline = 'bayestar'

# setting enclosed probability percentage 
prob_percentage = 0.9

# loading the simulated CBC event id (2015)
from astropy.utils.data import download_file

url_id = 'http://www.ligo.org/scientists/first2years/2015/compare/'+event_id+'/'+skymap_pipeline+'.fits.gz'
pipeline_event = download_file( url_id, cache = True, timeout = 300 )

# sending to Aladin plane
send_file ( pipeline_event )
rename ( skymap_pipeline + event_id )

# plotting contours from a specific url
from mocpy import MOC
get_json_link( 'https://losc.ligo.org/s/skymapViewer/json/skymaps/F2Y/'+event_id+'.json' )

#slider MOC production
interact( MOC_confidence_region, infile = pipeline_event, percentage = (0.1, 0.9, 0.1), 
         short_name = fixed( event_id ) )

# loading DSS colored for sky background
get_hips( "P/DSS2/color" )

**See the completed and updated results in the section [Sky map visualization with Aladin](#1.A-Sky-map-visualization-with-Aladin)**